In [ ]:
import numpy as np
import pickle
#%matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tifffile as tiff
from libtiff import TIFF

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.mixture import BayesianGaussianMixture
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier

import keras
from keras.utils import to_categorical
from keras.layers import Dense, Input, Conv2D, Dropout, Permute, Reshape, Flatten, GaussianNoise, MaxPooling2D, Concatenate
from keras.models import Sequential, Model
from keras.engine.topology import Input
from keras.utils import plot_model


pickle_save_path = "../../Data/"
pickle_col_augmented_path = "../../Data/col_augmented.pickle"
pickle_col_not_augmented_path = "../../Data/col_not_augmented.pickle"
pickle_grey_augmented_path = "../../Data/grey_augmented.pickle"
pickle_grey_not_augmented_path = "../../Data/grey_not_augmented.pickle"
pickle_grey_augmented_binary_save_path = "../../Data/grey_augmented_binary.pickle"
pickle_hough_val_save_path = "../../Data/hough_val.pickle"
pickle_hog_val_save_path = "../../Data/hog_val.pickle"
pickle_hes_val_save_path = "../../Data/hes_val.pickle"

### CNN directement sur image couleur

In [ ]:
with open(pickle_col_augmented_path, 'rb') as my_pickle:
    X, Y = pickle.load(my_pickle)
#X_norm = normalize(X.reshape((X.shape[0],-1))) #we normalize the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [ ]:
prob_dropout = 0.3
model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = 3, activation ='relu', data_format = "channels_first", input_shape = (3,31,31)))
model.add(Dropout(prob_dropout))
model.add(MaxPooling2D(pool_size = (2,2), data_format = "channels_first"))
model.add(Conv2D(filters = 32, kernel_size = 3, activation ='relu', data_format = "channels_first"))
model.add(Dropout(prob_dropout))
model.add(MaxPooling2D(pool_size = (2,2), data_format = "channels_first"))
model.add(Conv2D(filters = 64, kernel_size = 3, activation ='relu', data_format = "channels_first"))
model.add(Dropout(prob_dropout))
model.add(MaxPooling2D(pool_size = (2,2), data_format = "channels_first"))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
plot_model(model, to_file='../../Images/model_cnn_img_coul.png')

Si l'apprentissage n'est pas fini --> relancer le .fit en faisant varier le nombre d'epochs.

In [ ]:
model.fit(X_train, Y_train,
          epochs=15,
          batch_size=64)

In [ ]:
score = model.evaluate(X_test, Y_test)
print(score)

### TEST CNN sur image couleur

In [ ]:
SIZE_IMAGETTE = 31
RAYON_IMAGETTE = int(SIZE_IMAGETTE/2)

def search_on_big_image(modele,image_path, need_preprocess = False, stride = 25, from_greyscale = False, zone = None, display_big = False, threshold = 0.5):
  #On fait glisser une fenêtre sur toute l'image
  imagettes_new = []
  tif_file = TIFF.open(image_path, mode='r')
  big_image = tif_file.read_image()
  if zone:
    pt1 = zone[0]
    pt2 = zone[1]
    if from_greyscale:
        big_image = big_image[pt1[0]:pt2[0]+1,pt1[1]:pt2[1]+1]
    else:
        big_image = big_image[:,pt1[0]:pt2[0]+1,pt1[1]:pt2[1]+1]
  big_shape = big_image.shape
  print("shape big image :",big_shape)
  if display_big:
    plt.ion()
    fig = plt.figure(figsize=(18, 18))
    ax = fig.add_subplot(111)
    if from_greyscale:
        ax.imshow(big_image, cmap='gray')
        plt.show()
    else:
        ax.imshow(np.moveaxis(big_image,0,2))
        fig.canvas.draw()
  nb_test = 0
  for xCoord in np.arange(start = SIZE_IMAGETTE, stop = big_shape[1] - SIZE_IMAGETTE, step = stride):
      for yCoord in np.arange(start = SIZE_IMAGETTE, stop = big_shape[2] - SIZE_IMAGETTE, step = stride):
            nb_test +=1
            if from_greyscale:
                test_imagette = big_image[xCoord-RAYON_IMAGETTE : xCoord+RAYON_IMAGETTE+1, yCoord-RAYON_IMAGETTE : yCoord+RAYON_IMAGETTE+1]                    
            else:
                test_imagette_color = big_image[:, xCoord-RAYON_IMAGETTE : xCoord+RAYON_IMAGETTE+1, yCoord-RAYON_IMAGETTE : yCoord+RAYON_IMAGETTE+1]
                #if grey_modele:
                #   test_imagette = RGBToGreyscale(test_imagette_color)
                if need_preprocess == True:
                    test_imagette = preprocessing_func(test_imagette_color)
                else:
                    test_imagette = test_imagette_color
            test_imagette = np.reshape(test_imagette,(1,) + test_imagette.shape)
            proba = modele.predict(test_imagette)[0]
            if proba > threshold:
                if from_greyscale:
                    imagettes_new.append(test_imagette)
                else:
                    imagettes_new.append(test_imagette_color)
                if display_big:
                    ax.scatter(yCoord, xCoord, s=10, c='white', marker='x')
                    ax.annotate(str(round(proba[0],2)), xy=(yCoord, xCoord), xytext=(yCoord, xCoord+2))
                    fig.canvas.draw()
  print(str(nb_test) + ' images testées')
  return imagettes_new

In [ ]:
#Image jamais utilisé :
#big_image_path = "../../DATA_MARNIERES/nuit_couleur/Axe3_06h05_14_NC.tif"
#imagettes_color = search_on_big_image(model, big_image_path,
#                                      zone = ([6036,10],[6792,734]), threshold = 0.9, display_big = True, stride=5)
#imagettes_color = search_on_big_image(model, big_image_path,
#                                      zone = ([6186,110],[6342,534]), threshold = 0.9, display_big = True, stride=5)

#big_image_path = "../../DATA_MARNIERES/nuit_couleur/Axe1_04h55_24_NC.tif"
#imagettes_color = search_on_big_image(model, big_image_path,
#                                      zone = ([7789,603],[8038,933]), threshold = 0.9, display_big = True, stride=5)

big_image_path = "../../DATA_MARNIERES/nuit_couleur/Axe4_06h10_14_NC.tif"
imagettes_color = search_on_big_image(model, big_image_path,
                                      zone = ([5744,230],[6240,1100]), threshold = 0.9, display_big = True, stride=5)

In [ ]:
len(imagettes_color)

In [ ]:
nb_imagettes = len(imagettes_color)
if nb_imagettes > 100:
    nb_imagettes = 100
    imagettes_color_reduce = imagettes_color[:nb_imagettes]
else:
    imagettes_color_reduce = imagettes_color
nb_col = 9
nb_row = int(nb_imagettes // nb_col) +1
pos = 0
plt.figure(figsize=(18, 18))
for imagette in imagettes_color_reduce:
    pos +=1
    plt.subplot(nb_row,nb_col,pos)
    plt.imshow(np.moveaxis(imagette,0,2))
plt.show()

### Find circles

In [ ]:
from skimage import data, color
from skimage.transform import hough_circle
from skimage.feature import peak_local_max, canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte
from skimage.color import rgb2grey

##### Calcul pour toute la base de données de la transformée de Hough

In [ ]:
with open(pickle_col_augmented_path, 'rb') as my_pickle:
    X, Y = pickle.load(my_pickle)

In [ ]:
X_hough = np.ndarray((X.shape[0],3))

In [ ]:
for i, image_col in enumerate(X):
    image = rgb2grey(np.moveaxis(image_col,0,2))
    edges = canny(image, sigma=1)
    hough_res = hough_circle(edges, hough_radii)

    centers = []
    accums = []
    radii = []

    for radius, h in zip(hough_radii, hough_res):
        # For each radius, extract two circles
        num_peaks = 2
        peaks = peak_local_max(h, num_peaks=num_peaks)
        centers.extend(peaks)
        accums.extend(h[peaks[:, 0], peaks[:, 1]])
        radii.extend([radius] * num_peaks)

    h_peaks = np.sort(accums)[::-1][:3]
    h_peaks = np.pad(h_peaks,3-len(h_peaks),mode = 'constant',constant_values=0)[:3]
    X_hough[i,:] = h_peaks

In [ ]:
pickle.dump([X_hough, Y], open(pickle_hough_val_save_path, 'wb' ))

## Etude des valeurs h peaks sur la base de données

In [ ]:
with open(pickle_col_augmented_path, 'rb') as my_pickle:
    X, Y = pickle.load(my_pickle)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [ ]:
X_pos = X[np.where(Y==1)[0]]
X_neg = X[np.where(Y==0)[0]]
print("Il y a {} positifs.".format(len(X_pos)))
print("Il y a {} negatifs.".format(len(X_neg)))

In [ ]:
from scipy.stats import describe
hough_radii = np.arange(15, 30, 2)
def find_h_peaks_descriptor(DataBaseCol):
    h_peaks_tot = []
    for image_col in DataBaseCol:
        image = rgb2grey(np.moveaxis(image_col,0,2))
        edges = canny(image, sigma=1)
        hough_res = hough_circle(edges, hough_radii)
        
        centers = []
        accums = []
        radii = []

        for radius, h in zip(hough_radii, hough_res):
            # For each radius, extract two circles
            num_peaks = 2
            peaks = peak_local_max(h, num_peaks=num_peaks)
            centers.extend(peaks)
            accums.extend(h[peaks[:, 0], peaks[:, 1]])
            radii.extend([radius] * num_peaks)
        
        h_peaks = np.sort(accums)[::-1][:3]
        h_peaks = np.pad(h_peaks,3-len(h_peaks),mode = 'constant',constant_values=0)[:3]
        h_peaks_tot.append(h_peaks)
    h_peaks_tot = np.array(h_peaks_tot)
    print(h_peaks_tot.shape)
    print("1")
    print(describe(h_peaks_tot[:,0]))
    print("2")
    print(describe(h_peaks_tot[:,1]))
    print("3")
    print(describe(h_peaks_tot[:,2]))
        
        

In [ ]:
find_h_peaks_descriptor(X_neg)

In [ ]:
find_h_peaks_descriptor(X_pos)

### CNN directement sur image couleur + Hough

In [ ]:
with open(pickle_col_augmented_path, 'rb') as my_pickle:
    X, Y = pickle.load(my_pickle)
    
with open(pickle_hough_val_save_path, 'rb') as my_pickle:
    X_hough, _ = pickle.load(my_pickle)

indices = np.arange(X.shape[0])
X_train, X_test, Y_train, Y_test, idx_train, idx_test = train_test_split(X, Y,indices, test_size=0.1, random_state=42)
X_hough_train = X_hough[idx_train,:]
X_hough_test = X_hough[idx_test,:]

In [ ]:
prob_dropout = 0.3
first_input = Input(shape=(3,31,31))
layer1 = Conv2D(filters = 16, kernel_size = 3, activation ='relu', data_format = "channels_first")(first_input)
layer1 = Dropout(prob_dropout)(layer1)
layer1 = MaxPooling2D(pool_size = (2,2), data_format = "channels_first")(layer1)
layer1 = Conv2D(filters = 32, kernel_size = 3, activation ='relu', data_format = "channels_first")(layer1)
layer1 = Dropout(prob_dropout)(layer1)
layer1 = MaxPooling2D(pool_size = (2,2), data_format = "channels_first")(layer1)
layer1 = Conv2D(filters = 64, kernel_size = 3, activation ='sigmoid', data_format = "channels_first")(layer1)
layer1 = Dropout(prob_dropout)(layer1)
layer1 = MaxPooling2D(pool_size = (2,2), data_format = "channels_first")(layer1)
layer1 = Flatten()(layer1)

second_input = Input(shape=(3,))
layer2 = Dense(3, activation='relu')(second_input)
merge_layer = Concatenate()([layer1, layer2])

merge_layer = Dense(64, activation='relu')(merge_layer)
merge_layer = Dense(32, activation='relu')(merge_layer)
merge_layer = Dense(1, activation='sigmoid')(merge_layer)

model = Model(inputs=[first_input, second_input], outputs=merge_layer)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
prob_dropout = 0.3
first_input = Input(shape=(3,31,31))
layer1 = Conv2D(filters = 16, kernel_size = 3, activation ='relu', data_format = "channels_first")(first_input)
layer1 = Dropout(prob_dropout)(layer1)
layer1 = MaxPooling2D(pool_size = (2,2), data_format = "channels_first")(layer1)
layer1 = Conv2D(filters = 32, kernel_size = 3, activation ='relu', data_format = "channels_first")(layer1)
layer1 = Dropout(prob_dropout)(layer1)
layer1 = MaxPooling2D(pool_size = (2,2), data_format = "channels_first")(layer1)
layer1 = Conv2D(filters = 64, kernel_size = 3, activation ='relu', data_format = "channels_first")(layer1)
layer1 = Dropout(prob_dropout)(layer1)
layer1 = MaxPooling2D(pool_size = (2,2), data_format = "channels_first")(layer1)
layer1 = Flatten()(layer1)

layer1 = Dense(64, activation='relu')(layer1)
layer1 = Dense(32, activation='sigmoid')(layer1)

second_input = Input(shape=(3,))
layer2 = Dense(3, activation='sigmoid')(second_input)

merge_layer = Concatenate()([layer1, layer2])

merge_layer = Dense(8, activation='relu')(merge_layer)
merge_layer = Dense(1, activation='sigmoid')(merge_layer)

model = Model(inputs=[first_input, second_input], outputs=merge_layer)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
plot_model(model, to_file='../../Images/model_cnn_img_coul_hough.png')

In [ ]:
model.fit([X_train ,X_hough_train], Y_train,
          epochs=12,
          batch_size=64)

In [ ]:
score = model.evaluate([X_test, X_hough_test], Y_test)
print(score)

#### Test CNN image couleur + hough

In [ ]:
SIZE_IMAGETTE = 31
RAYON_IMAGETTE = int(SIZE_IMAGETTE/2)

def preprocessing_func(img_col):
    hough_radii = np.arange(15, 30, 2)
    image = rgb2grey(np.moveaxis(img_col,0,2))
    edges = canny(image, sigma=1)
    hough_res = hough_circle(edges, hough_radii)

    centers = []
    accums = []
    radii = []

    for radius, h in zip(hough_radii, hough_res):
        # For each radius, extract two circles
        num_peaks = 2
        peaks = peak_local_max(h, num_peaks=num_peaks)
        centers.extend(peaks)
        accums.extend(h[peaks[:, 0], peaks[:, 1]])
        radii.extend([radius] * num_peaks)

    h_peaks = np.sort(accums)[::-1][:3]
    h_peaks = np.pad(h_peaks,3-len(h_peaks),mode = 'constant',constant_values=0)[:3]
    return h_peaks

def search_on_big_image(modele,image_path, need_preprocess = False, stride = 25, from_greyscale = False, zone = None, display_big = False, threshold = 0.5):
  #On fait glisser une fenêtre sur toute l'image
  imagettes_new = []
  tif_file = TIFF.open(image_path, mode='r')
  big_image = tif_file.read_image()
  if zone:
    pt1 = zone[0]
    pt2 = zone[1]
    if from_greyscale:
        big_image = big_image[pt1[0]:pt2[0]+1,pt1[1]:pt2[1]+1]
    else:
        big_image = big_image[:,pt1[0]:pt2[0]+1,pt1[1]:pt2[1]+1]
  big_shape = big_image.shape
  print("shape big image :",big_shape)
  if display_big:
    plt.ion()
    fig = plt.figure(figsize=(18, 18))
    ax = fig.add_subplot(111)
    if from_greyscale:
        ax.imshow(big_image, cmap='gray')
        plt.show()
    else:
        ax.imshow(np.moveaxis(big_image,0,2))
        fig.canvas.draw()
  nb_test = 0
  for xCoord in np.arange(start = SIZE_IMAGETTE, stop = big_shape[1] - SIZE_IMAGETTE, step = stride):
      for yCoord in np.arange(start = SIZE_IMAGETTE, stop = big_shape[2] - SIZE_IMAGETTE, step = stride):
            nb_test +=1
            if from_greyscale:
                test_imagette = big_image[xCoord-RAYON_IMAGETTE : xCoord+RAYON_IMAGETTE+1, yCoord-RAYON_IMAGETTE : yCoord+RAYON_IMAGETTE+1]                    
            else:
                test_imagette_color = big_image[:, xCoord-RAYON_IMAGETTE : xCoord+RAYON_IMAGETTE+1, yCoord-RAYON_IMAGETTE : yCoord+RAYON_IMAGETTE+1]
                #if grey_modele:
                #   test_imagette = RGBToGreyscale(test_imagette_color)
                if need_preprocess == True:
                    h_peaks = preprocessing_func(test_imagette_color)
                    test_imagette = test_imagette_color
                else:
                    test_imagette = test_imagette_color
            test_imagette = np.reshape(test_imagette,(1,) + test_imagette.shape)
            h_peaks = np.reshape(h_peaks,(1,) + h_peaks.shape)
            proba = modele.predict([test_imagette, h_peaks])[0]
            if proba > threshold:
                if from_greyscale:
                    imagettes_new.append(test_imagette)
                else:
                    imagettes_new.append(test_imagette_color)
                if display_big:
                    ax.scatter(yCoord, xCoord, s=10, c='white', marker='x')
                    ax.annotate(str(round(proba[0],2)), xy=(yCoord, xCoord), xytext=(yCoord, xCoord+2))
                    fig.canvas.draw()
  print(str(nb_test) + ' images testées')
  return imagettes_new

In [ ]:
#Image jamais utilisé :
big_image_path = "../../DATA_MARNIERES/nuit_couleur/Axe3_06h05_14_NC.tif"
#imagettes_color = search_on_big_image(model, big_image_path, need_preprocess = True,
#                                      zone = ([6036,10],[6792,734]), threshold = 0.9, display_big = True, stride=5)
#imagettes_color = search_on_big_image(model, big_image_path, need_preprocess = True,
#                                      zone = ([6186,110],[6342,534]), threshold = 0.9, display_big = True, stride=5)

#big_image_path = "../../DATA_MARNIERES/nuit_couleur/Axe1_04h55_24_NC.tif"
#imagettes_color = search_on_big_image(model, big_image_path, need_preprocess = True,
#                                      zone = ([7789,603],[8038,933]), threshold = 0.9, display_big = True, stride=5)

big_image_path = "../../DATA_MARNIERES/nuit_couleur/Axe4_06h10_14_NC.tif"
imagettes_color = search_on_big_image(model, big_image_path,need_preprocess = True,
                                      zone = ([5744,230],[6240,1100]), threshold = 0.9, display_big = True, stride=5)

In [ ]:
len(imagettes_color)

In [ ]:
nb_imagettes = len(imagettes_color)
if nb_imagettes > 100:
    nb_imagettes = 100
    imagettes_color_reduce = imagettes_color[:nb_imagettes]
else:
    imagettes_color_reduce = imagettes_color
nb_col = 9
nb_row = int(nb_imagettes // nb_col) +1
pos = 0
plt.figure(figsize=(18, 18))
for imagette in imagettes_color_reduce:
    pos +=1
    plt.subplot(nb_row,nb_col,pos)
    plt.imshow(np.moveaxis(imagette,0,2))
plt.show()

### matrice hessian ?

In [ ]:
from skimage.feature import hessian_matrix

#### Calcul sur toute la base

In [ ]:
with open(pickle_col_augmented_path, 'rb') as my_pickle:
    X, Y = pickle.load(my_pickle)

In [ ]:
X_hes = np.ndarray((X.shape))

In [ ]:
for i, image_col in enumerate(X):
    image = rgb2grey(np.moveaxis(image_col,0,2))
    X_hes[i] = np.array(hessian_matrix(image,order="rc", sigma=1))

In [ ]:
pickle.dump([X_hes, Y], open(pickle_hes_val_save_path, 'wb' ))

### CNN sur hessian value

In [ ]:
with open(pickle_hes_val_save_path, 'rb') as my_pickle:
    X, Y = pickle.load(my_pickle)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [ ]:
prob_dropout = 0.3
model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = 3, activation ='relu', data_format = "channels_first", input_shape = (3,31,31)))
model.add(Dropout(prob_dropout))
model.add(MaxPooling2D(pool_size = (2,2), data_format = "channels_first"))
model.add(Conv2D(filters = 32, kernel_size = 3, activation ='relu', data_format = "channels_first"))
model.add(Dropout(prob_dropout))
model.add(MaxPooling2D(pool_size = (2,2), data_format = "channels_first"))
model.add(Conv2D(filters = 64, kernel_size = 3, activation ='relu', data_format = "channels_first"))
model.add(Dropout(prob_dropout))
model.add(MaxPooling2D(pool_size = (2,2), data_format = "channels_first"))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train, Y_train,
          epochs=15,
          batch_size=64)

In [ ]:
score = model.evaluate(X_test, Y_test)
print(score)

### TEST CNN sur hessian value

In [ ]:
SIZE_IMAGETTE = 31
RAYON_IMAGETTE = int(SIZE_IMAGETTE/2)

def preprocessing_func(image_col):
    image = rgb2grey(np.moveaxis(image_col,0,2))
    return np.array(hessian_matrix(image,order="rc", sigma=1))

def search_on_big_image(modele,image_path, need_preprocess = False, stride = 25, from_greyscale = False, zone = None, display_big = False, threshold = 0.5):
  #On fait glisser une fenêtre sur toute l'image
  imagettes_new = []
  tif_file = TIFF.open(image_path, mode='r')
  big_image = tif_file.read_image()
  if zone:
    pt1 = zone[0]
    pt2 = zone[1]
    if from_greyscale:
        big_image = big_image[pt1[0]:pt2[0]+1,pt1[1]:pt2[1]+1]
    else:
        big_image = big_image[:,pt1[0]:pt2[0]+1,pt1[1]:pt2[1]+1]
  big_shape = big_image.shape
  print("shape big image :",big_shape)
  if display_big:
    plt.ion()
    fig = plt.figure(figsize=(18, 18))
    ax = fig.add_subplot(111)
    if from_greyscale:
        ax.imshow(big_image, cmap='gray')
        plt.show()
    else:
        ax.imshow(np.moveaxis(big_image,0,2))
        fig.canvas.draw()
  nb_test = 0
  for xCoord in np.arange(start = SIZE_IMAGETTE, stop = big_shape[1] - SIZE_IMAGETTE, step = stride):
      for yCoord in np.arange(start = SIZE_IMAGETTE, stop = big_shape[2] - SIZE_IMAGETTE, step = stride):
            nb_test +=1
            if from_greyscale:
                test_imagette = big_image[xCoord-RAYON_IMAGETTE : xCoord+RAYON_IMAGETTE+1, yCoord-RAYON_IMAGETTE : yCoord+RAYON_IMAGETTE+1]                    
            else:
                test_imagette_color = big_image[:, xCoord-RAYON_IMAGETTE : xCoord+RAYON_IMAGETTE+1, yCoord-RAYON_IMAGETTE : yCoord+RAYON_IMAGETTE+1]
                #if grey_modele:
                #   test_imagette = RGBToGreyscale(test_imagette_color)
                if need_preprocess == True:
                    test_imagette = preprocessing_func(test_imagette_color)
                else:
                    test_imagette = test_imagette_color
            test_imagette = np.reshape(test_imagette,(1,) + test_imagette.shape)
            proba = modele.predict(test_imagette)[0]
            if proba > threshold:
                if from_greyscale:
                    imagettes_new.append(test_imagette)
                else:
                    imagettes_new.append(test_imagette_color)
                if display_big:
                    ax.scatter(yCoord, xCoord, s=10, c='white', marker='x')
                    ax.annotate(str(round(proba[0],2)), xy=(yCoord, xCoord), xytext=(yCoord, xCoord+2))
                    fig.canvas.draw()
  print(str(nb_test) + ' images testées')
  return imagettes_new

In [ ]:
#Image jamais utilisé :
big_image_path = "../../DATA_MARNIERES/nuit_couleur/Axe3_06h05_14_NC.tif"
imagettes_color = search_on_big_image(model, big_image_path,need_preprocess=True,
                                      zone = ([6036,10],[6792,734]), threshold = 0.8, display_big = True, stride=5)
#imagettes_color = search_on_big_image(model, big_image_path, need_preprocess=True,
#                                      zone = ([6186,110],[6342,534]), threshold = 0.7, display_big = True, stride=5)

In [ ]:
len(imagettes_color)

In [ ]:
nb_imagettes = len(imagettes_color)
if nb_imagettes > 100:
    nb_imagettes = 100
    imagettes_color_reduce = imagettes_color[:nb_imagettes]
else:
    imagettes_color_reduce = imagettes_color
nb_col = 9
nb_row = int(nb_imagettes // nb_col) +1
pos = 0
plt.figure(figsize=(18, 18))
for imagette in imagettes_color_reduce:
    pos +=1
    plt.subplot(nb_row,nb_col,pos)
    plt.imshow(np.moveaxis(imagette,0,2))
plt.show()

## CNN sur hessian value + hough

In [ ]:
with open(pickle_hes_val_save_path, 'rb') as my_pickle:
    X_hes, Y = pickle.load(my_pickle)
    
with open(pickle_hough_val_save_path, 'rb') as my_pickle:
    X_hough, _ = pickle.load(my_pickle)

indices = np.arange(X_hes.shape[0])
X_train, X_test, Y_train, Y_test, idx_train, idx_test = train_test_split(X_hes, Y, indices, test_size=0.1, random_state=42)
X_hough_train = X_hough[idx_train,:]
X_hough_test = X_hough[idx_test,:]

In [ ]:
prob_dropout = 0.3
first_input = Input(shape=(3,31,31))
layer1 = Conv2D(filters = 16, kernel_size = 3, activation ='relu', data_format = "channels_first")(first_input)
layer1 = Dropout(prob_dropout)(layer1)
layer1 = MaxPooling2D(pool_size = (2,2), data_format = "channels_first")(layer1)
layer1 = Conv2D(filters = 32, kernel_size = 3, activation ='relu', data_format = "channels_first")(layer1)
layer1 = Dropout(prob_dropout)(layer1)
layer1 = MaxPooling2D(pool_size = (2,2), data_format = "channels_first")(layer1)
layer1 = Conv2D(filters = 64, kernel_size = 3, activation ='relu', data_format = "channels_first")(layer1)
layer1 = Dropout(prob_dropout)(layer1)
layer1 = MaxPooling2D(pool_size = (2,2), data_format = "channels_first")(layer1)
layer1 = Flatten()(layer1)

layer1 = Dense(64, activation='relu')(layer1)
layer1 = Dense(32, activation='sigmoid')(layer1)

second_input = Input(shape=(3,))
layer2 = Dense(3, activation='sigmoid')(second_input)

merge_layer = Concatenate()([layer1, layer2])

merge_layer = Dense(8, activation='relu')(merge_layer)
merge_layer = Dense(1, activation='sigmoid')(merge_layer)

model = Model(inputs=[first_input, second_input], outputs=merge_layer)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
model.fit([X_train ,X_hough_train], Y_train,
          epochs=2,
          batch_size=64)

In [ ]:
score = model.evaluate([X_test, X_hough_test], Y_test)
print(score)

#### Test CNN hessian value + hough

In [ ]:
SIZE_IMAGETTE = 31
RAYON_IMAGETTE = int(SIZE_IMAGETTE/2)

def preprocessing_hough_func(img_col):
    hough_radii = np.arange(15, 30, 2)
    image = rgb2grey(np.moveaxis(img_col,0,2))
    edges = canny(image, sigma=1)
    hough_res = hough_circle(edges, hough_radii)

    centers = []
    accums = []
    radii = []

    for radius, h in zip(hough_radii, hough_res):
        # For each radius, extract two circles
        num_peaks = 2
        peaks = peak_local_max(h, num_peaks=num_peaks)
        centers.extend(peaks)
        accums.extend(h[peaks[:, 0], peaks[:, 1]])
        radii.extend([radius] * num_peaks)

    h_peaks = np.sort(accums)[::-1][:3]
    h_peaks = np.pad(h_peaks,3-len(h_peaks),mode = 'constant',constant_values=0)[:3]
    return h_peaks

def preprocessing_hes_func(image_col):
    image = rgb2grey(np.moveaxis(image_col,0,2))
    return np.array(hessian_matrix(image,order="rc", sigma=1))

def search_on_big_image(modele,image_path, need_preprocess = False, stride = 25, from_greyscale = False, zone = None, display_big = False, threshold = 0.5):
  #On fait glisser une fenêtre sur toute l'image
  imagettes_new = []
  tif_file = TIFF.open(image_path, mode='r')
  big_image = tif_file.read_image()
  if zone:
    pt1 = zone[0]
    pt2 = zone[1]
    if from_greyscale:
        big_image = big_image[pt1[0]:pt2[0]+1,pt1[1]:pt2[1]+1]
    else:
        big_image = big_image[:,pt1[0]:pt2[0]+1,pt1[1]:pt2[1]+1]
  big_shape = big_image.shape
  print("shape big image :",big_shape)
  if display_big:
    plt.ion()
    fig = plt.figure(figsize=(18, 18))
    ax = fig.add_subplot(111)
    if from_greyscale:
        ax.imshow(big_image, cmap='gray')
        plt.show()
    else:
        ax.imshow(np.moveaxis(big_image,0,2))
        fig.canvas.draw()
  nb_test = 0
  for xCoord in np.arange(start = SIZE_IMAGETTE, stop = big_shape[1] - SIZE_IMAGETTE, step = stride):
      for yCoord in np.arange(start = SIZE_IMAGETTE, stop = big_shape[2] - SIZE_IMAGETTE, step = stride):
            nb_test +=1
            if from_greyscale:
                test_imagette = big_image[xCoord-RAYON_IMAGETTE : xCoord+RAYON_IMAGETTE+1, yCoord-RAYON_IMAGETTE : yCoord+RAYON_IMAGETTE+1]                    
            else:
                test_imagette_color = big_image[:, xCoord-RAYON_IMAGETTE : xCoord+RAYON_IMAGETTE+1, yCoord-RAYON_IMAGETTE : yCoord+RAYON_IMAGETTE+1]
                #if grey_modele:
                #   test_imagette = RGBToGreyscale(test_imagette_color)
                if need_preprocess == True:
                    h_peaks = preprocessing_hough_func(test_imagette_color)
                    test_hes = preprocessing_hes_func(test_imagette_color)
                else:
                    test_imagette = test_imagette_color
            test_hes = np.reshape(test_hes,(1,) + test_hes.shape)
            h_peaks = np.reshape(h_peaks,(1,) + h_peaks.shape)
            proba = modele.predict([test_hes, h_peaks])[0]
            if proba > threshold:
                if from_greyscale:
                    imagettes_new.append(test_imagette)
                else:
                    imagettes_new.append(test_imagette_color)
                if display_big:
                    ax.scatter(yCoord, xCoord, s=10, c='white', marker='x')
                    ax.annotate(str(round(proba[0],2)), xy=(yCoord, xCoord), xytext=(yCoord, xCoord+2))
                    fig.canvas.draw()
  print(str(nb_test) + ' images testées')
  return imagettes_new

In [ ]:
#Image jamais utilisé :
big_image_path = "../../DATA_MARNIERES/nuit_couleur/Axe3_06h05_14_NC.tif"
#imagettes_color = search_on_big_image(model, big_image_path, need_preprocess = True,
#                                      zone = ([6036,10],[6792,734]), threshold = 0.9, display_big = True, stride=5)
imagettes_color = search_on_big_image(model, big_image_path, need_preprocess = True,
                                     zone = ([6186,110],[6342,534]), threshold = 0.9, display_big = True, stride=5)

#big_image_path = "../../DATA_MARNIERES/nuit_couleur/Axe1_04h55_24_NC.tif"
#imagettes_color = search_on_big_image(model, big_image_path, need_preprocess = True,
#                                      zone = ([7789,603],[8038,933]), threshold = 0.9, display_big = True, stride=5)

#big_image_path = "../../DATA_MARNIERES/nuit_couleur/Axe4_06h10_14_NC.tif"
#imagettes_color = search_on_big_image(model, big_image_path,need_preprocess = True,
#                                      zone = ([5744,230],[6240,1100]), threshold = 0.9, display_big = True, stride=5)

In [ ]:
len(imagettes_color)

In [ ]:
nb_imagettes = len(imagettes_color)
if nb_imagettes > 100:
    nb_imagettes = 100
    imagettes_color_reduce = imagettes_color[:nb_imagettes]
else:
    imagettes_color_reduce = imagettes_color
nb_col = 9
nb_row = int(nb_imagettes // nb_col) +1
pos = 0
plt.figure(figsize=(18, 18))
for imagette in imagettes_color_reduce:
    pos +=1
    plt.subplot(nb_row,nb_col,pos)
    plt.imshow(np.moveaxis(imagette,0,2))
plt.show()